In [1]:
import requests
import json
import pandas as pd
#import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv
import math
#import descartes
#import geoplot
#import plotly.figure_factory as ff
import scipy.stats as st
%reload_ext autoreload
%autoreload 2
import modules

In [2]:
pop_df = modules.census_api_data_cleaner()
rural_lookup = modules.rural_county_info_cleaner('County_Rural_Lookup.xlsx')
binned_pop_df = modules.classify_county_populations(pop_df, rural_lookup)

In [4]:
employment_df = modules.income_unemployment_cleaner('Unemployment.csv')

In [6]:
county_df = binned_pop_df.merge(employment_df[['FIPS', 'Unemployment_rate_2017', 'Median_Household_Income_2017']], on = 'FIPS', how = 'inner')
county_df.head()

,POP,FIPS,County_State,Percent_Rural,pop_category,Unemployment_rate_2017,Median_Household_Income_2017
0,55601,01001,"Autauga County, Alabama",42.002162,Mix/Suburban,3,58343
1,218022,01003,"Baldwin County, Alabama",42.279099,Mix/Suburban,4,56607
2,24881,01005,"Barbour County, Alabama",67.789635,Mix/Suburban,5,32490
3,22400,01007,"Bibb County, Alabama",68.352607,Mix/Suburban,4,45795
4,57840,01009,"Blount County, Alabama",89.951502,Rural,4,48253
